In [5]:

import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.applications.vgg16 import VGG16
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# 1 Baseline CNN Model


In [6]:

def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = 'Baseline_CNN'
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
    
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Train/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Test/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=2)
	# evaluate model
	_, acc = model.evaluate(test_it, steps=len(test_it), verbose=2)
	print('> %.4f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
    
run_test_harness()

C:\Users\Sresth\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/20
1/1 - 5s - loss: 0.7173 - accuracy: 0.5167 - val_loss: 0.7403 - val_accuracy: 0.5000 - 5s/epoch - 5s/step
Epoch 2/20
1/1 - 3s - loss: 0.6669 - accuracy: 0.5000 - val_loss: 1.4711 - val_accuracy: 0.5000 - 3s/epoch - 3s/step
Epoch 3/20
1/1 - 3s - loss: 1.2767 - accuracy: 0.5000 - val_loss: 0.8328 - val_accuracy: 0.5000 - 3s/epoch - 3s/step
Epoch 4/20
1/1 - 3s - loss: 0.7687 - accuracy: 0.5000 - val_loss: 0.7649 - val_accuracy: 0.5000 - 3s/epoch - 3s/step
Epoch 5/20
1/1 - 3s - loss: 0.6951 - accuracy: 0.5000 - val_loss: 0.6113 - val_accuracy: 0.6500 - 3s/epoch - 3s/step
Epoch 6/20
1/1 - 3s - loss: 0.4898 - accuracy: 0.9167 - val_loss: 0.7295 - val_accuracy: 0.5500 - 3s/epoch - 3s/step
Epoch 7/20
1/1 - 3s - loss: 0.5551 - accuracy: 0.6667 - val_loss: 0.4867 - val_accuracy: 0.8000 - 3s/epoch - 3s/step
Epoch 8/20
1/1 - 3s - loss: 0.3603 - accuracy: 0.9667 - val_loss: 0.5200 - val_accuracy: 0.6500 - 3s/

# Transfer Learning with VGG16

In [7]:
# define cnn model
from keras.models import Model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = 'VGG16'
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	test_it = datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Test/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
	# evaluate model
	_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
    
run_test_harness()
 

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/10
1/1 [==============================] - 25s 25s/step - loss: 11.6735 - accuracy: 0.3333 - val_loss: 18.6653 - val_accuracy: 0.5500
Epoch 2/10
1/1 [==============================] - 23s 23s/step - loss: 8.1516 - accuracy: 0.7167 - val_loss: 1.3690e-07 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 23s 23s/step - loss: 0.4956 - accuracy: 0.9833 - val_loss: 1.9574e-26 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 23s 23s/step - loss: 0.8204 - accuracy: 0.9833 - val_loss: 6.2953e-25 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 23s 23s/step - loss: 0.2832 - accuracy: 0.9833 - val_loss: 9.1385e-31 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 23s 23s/step - loss: 9.9686e-10 - accuracy: 1.0000 - val_loss: 1.5159e-36 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 67s 67s/st

# 3 block model with image data augmentation

In [8]:

def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = 'Data Augmentation'
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generators
	train_datagen = ImageDataGenerator(rescale=1.0/255.0,
		width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
	test_datagen = ImageDataGenerator(rescale=1.0/255.0)
	# prepare iterators
	train_it = train_datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Train/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	test_it = test_datagen.flow_from_directory(r'C:\Users\Sresth\Desktop\Sem 6 material\ImageClassifier\Data\Test/',
		class_mode='binary', batch_size=64, target_size=(200, 200))
	# fit model
	history = model.fit(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=0)
	# evaluate model
	_, acc = model.evaluate(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
    
run_test_harness()


Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
> 90.000


In [3]:
cd Sem 6 material

C:\Users\Sresth\desktop\Sem 6 material
